In [1]:
from transformers import * 
import tensorflow as tf 
from tensorflow.keras.layers import * 
from tensorflow.keras.models import * 
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.text import *  
from tensorflow.keras.losses import * 
from tensorflow.keras.optimizers import *
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import random 
import re 
import math 
from tqdm import tqdm 
import sklearn
from official import nlp
import official.nlp.optimization 
import official.nlp.modeling.losses 
import official.nlp.modeling.models 
import official.nlp.modeling.networks

import torch 
import torch.nn as nn 
import torch.functional as f
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
import time
import datetime

PyTorch version 1.4.0 available.
TensorFlow version 2.5.0 available.
/usr/local/lib/python3.6/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.5.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


# Preprocess Data

- Sliding window split 
- Tokenization (Head+Tail if too long) 

In [2]:
train = pd.read_csv("open/train.csv") 
test = pd.read_csv("open/test.csv") 

train.shape, test.shape, train['label'].nunique()

((174304, 13), (43576, 12), 46)

In [3]:
num_classes = train['label'].nunique() # 46 

In [4]:
submission = pd.read_csv("open/sample_submission.csv") 

In [5]:
for column in train.columns: 
    print(column)

index
제출년도
사업명
사업_부처명
계속과제여부
내역사업명
과제명
요약문_연구목표
요약문_연구내용
요약문_기대효과
요약문_한글키워드
요약문_영문키워드
label


In [6]:
def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
    return sent_clean

In [7]:
def split_text(s, overlap = 20, chunk_size = 50): 
    total = [] 
    partial = [] 
    if len(s.split()) // (chunk_size - overlap) > 0:  
        n = len(s.split()) // (chunk_size - overlap) 
    else: 
        n = 1 
    for w in range(n): 
        if w == 0: 
            partial = s.split()[:chunk_size] 
            total.append(" ".join(partial)) 
        else:  
            partial = s.split()[w*(chunk_size - overlap):w*(chunk_size - overlap) + chunk_size]
            total.append(" ".join(partial)) 
    return total

In [8]:
train['요약문_내용'] = train['요약문_연구목표'] + train['요약문_연구내용'] + train['요약문_기대효과'] 
test['요약문_내용'] = test['요약문_연구목표'] + test['요약문_연구내용'] + test['요약문_기대효과'] 

In [9]:
train['요약문_내용'].fillna('NAN',inplace=True) 
test['요약문_내용'].fillna('NAN',inplace=True)

In [10]:
train['사업명'].fillna('NAN',inplace=True) 
train['사업_부처명'].fillna('NAN',inplace=True) 
train['내역사업명'].fillna('NAN',inplace=True) 
train['과제명'].fillna('NAN',inplace=True) 
train['요약문_한글키워드'].fillna('NAN',inplace=True)

In [11]:
contents = train['요약문_내용'].values 
feature1 = train['사업명'].values 
feature2 = train['사업_부처명'].values 
feature3 = train['내역사업명'].values 
feature4 = train['과제명'].values 
feature5 = train['요약문_한글키워드'].values 
feature6 = train['label'].values 

train_data = {'사업명':[],'사업_부처명':[],'내역사업명':[],'과제명':[],'한글키워드':[],'요약문':[],'label':[]} 

for i in tqdm(range(contents.shape[0]), position = 0, leave = True): 
    sample = str(contents[i]) 
    splitted_text = split_text(clean_text(sample)) 
    for t in splitted_text: 
        train_data['요약문'].append(t) 
        train_data['사업명'].append(clean_text(str(feature1[i])))
        train_data['사업_부처명'].append(clean_text(str(feature2[i]))) 
        train_data['내역사업명'].append(clean_text(str(feature3[i]))) 
        train_data['과제명'].append(clean_text(str(feature4[i])))  
        train_data['한글키워드'].append(feature5[i]) # no cleaning for this one
        train_data['label'].append(feature6[i])  

100%|██████████| 174304/174304 [01:43<00:00, 1687.84it/s]


In [12]:
train_data = pd.DataFrame(train_data)

In [13]:
train_data.head(5) 

,사업명,사업_부처명,내역사업명,과제명,한글키워드,요약문,label
0,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",새로운 해충분류군의 동정기술 개발 및 유입확산 추적 가 외래 및 돌발해충의 발생조사...,24
1,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",의 돌발 및 외래해충 다 외래 및 돌발해충의 유전적 다양성 조사 시험곤충 나 의 해...,24
2,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",최종목표 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구 발굴된 유전자를 통한 ...,0
3,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",저항성 극복 기전을 규명 환자조직 동물실험 세포실험을 통해 대장암에 특이적으로 조절...,0
4,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",내성 유발 유전자를 발굴하고 저항성 극복 기전을 규명 추후 기반 항암화학요법 치료효...,0


In [14]:
## Now we tokenize each data and make sure they all lie within the 512 tokenization range 
## if not check how many have token length greater than 512 

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator") 

def electra_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer.encode_plus(
        text = sent, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences, not "really" necessary for now    
    
    if len(input_id) > 512: # head + tail methodology 
        input_id = input_id[:129] + input_id[-383:] 
        attention_mask = attention_mask[:129] + attention_mask[-383:]  
        token_type_id = token_type_id[:129] + token_type_id[-383:]    
        print("Long Text!! Using Head+Tail Truncation")
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        token_type_id = token_type_id + [0]*(512 - len(token_type_id))  
        
    return input_id, attention_mask, token_type_id


In [15]:
train_data['data'] = train_data['사업명'] + " " + train_data['사업_부처명'] + " " + train_data['내역사업명'] + " " + train_data['과제명'] + " " + train_data['한글키워드'] + " " + train_data['요약문'] 

train_data.head(2)


,사업명,사업_부처명,내역사업명,과제명,한글키워드,요약문,label,data
0,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",새로운 해충분류군의 동정기술 개발 및 유입확산 추적 가 외래 및 돌발해충의 발생조사...,24,농업기초기반연구 농촌진흥청 농산물안전성연구 유전정보를 활용한 새로운 해충 분류군 동...
1,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",의 돌발 및 외래해충 다 외래 및 돌발해충의 유전적 다양성 조사 시험곤충 나 의 해...,24,농업기초기반연구 농촌진흥청 농산물안전성연구 유전정보를 활용한 새로운 해충 분류군 동...


In [16]:
train_text = train_data['data'].values 
train_labels = train_data['label'].values

In [17]:
BATCH_SIZE = 32
NUM_EPOCHS = 30
VALID_SPLIT = 0.1 
MAX_LEN = 512 # max token size for BERT, ELECTRA

In [18]:
N = train_data.shape[0] 

input_ids = np.zeros((N, MAX_LEN),dtype=int)
attention_masks = np.zeros((N, MAX_LEN),dtype=int)
token_type_ids = np.zeros((N, MAX_LEN),dtype=int) 
labels = np.zeros((N),dtype=int)

for i in tqdm(range(N), position=0, leave=True): 
    try:
        cur_str = train_text[i]
        cur_label = train_labels[i]
        input_id, attention_mask, token_type_id = electra_tokenizer(cur_str, MAX_LEN=MAX_LEN) 
        input_ids[i,] = input_id 
        attention_masks[i,] = attention_mask 
        token_type_ids[i,] = token_type_id
        labels[i] = cur_label 
    except Exception as e: 
        print(e)
        print(cur_str)
        pass

  5%|▍         | 79935/1638867 [02:19<41:02, 633.04it/s]

Long Text!! Using Head+Tail Truncation


  8%|▊         | 122916/1638867 [03:32<37:53, 666.76it/s]  

Long Text!! Using Head+Tail Truncation


  8%|▊         | 138481/1638867 [03:58<39:24, 634.65it/s]  

...

 95%|█████████▍| 1554788/1638867 [44:54<02:21, 594.74it/s]

Long Text!! Using Head+Tail Truncation


 96%|█████████▌| 1573388/1638867 [45:30<01:40, 650.58it/s]

Long Text!! Using Head+Tail Truncation


100%|██████████| 1638867/1638867 [47:24<00:00, 576.21it/s]


163만개중 100개 미만이 long text인 것 같다. 나쁘지 않은듯

In [19]:
input_ids = torch.tensor(input_ids, dtype=int)

In [20]:
attention_masks = torch.tensor(attention_masks, dtype=int)

In [21]:
token_type_ids = torch.tensor(token_type_ids, dtype=int) 

labels = torch.tensor(labels, dtype=int)

In [22]:
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, random_state = 42, test_size = VALID_SPLIT) 

train_attention_mask, val_attention_mask, _, _ = train_test_split(attention_masks, labels, random_state = 42, test_size = VALID_SPLIT) 

train_token_ids, val_token_ids, _, _ = train_test_split(token_type_ids, labels, random_state = 42, test_size = VALID_SPLIT) 


train_inputs.shape, train_attention_mask.shape, train_token_ids.shape, train_labels.shape

(torch.Size([1474980, 512]),
 torch.Size([1474980, 512]),
 torch.Size([1474980, 512]),
 torch.Size([1474980]))

In [23]:
val_inputs.shape, val_attention_mask.shape, val_token_ids.shape, val_labels.shape

(torch.Size([163887, 512]),
 torch.Size([163887, 512]),
 torch.Size([163887, 512]),
 torch.Size([163887]))

In [24]:
batch_size = 32 
train_data = TensorDataset(train_inputs, train_attention_mask, train_token_ids, train_labels) 
train_sampler = RandomSampler(train_data) 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size) 

validation_data = TensorDataset(val_inputs, val_attention_mask, val_token_ids, val_labels) 
validation_sampler = SequentialSampler(validation_data) 
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size) 

In [25]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=46)
model.cuda()

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [26]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

epochs = 30 

total_steps = len(train_dataloader) * epochs 

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps) 

In [27]:
def flat_accuracy(preds, labels): 
    pred_flat = np.argmax(preds, axis=1).flatten() 
    labels_flat = labels.flatten() 
    return np.sum(pred_flat == labels_flat) / len(labels_flat) 

def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [28]:
device = torch.device("cuda")

In [29]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 20 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            print('  current average loss = {}'.format(total_loss / step))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=b_token_type_ids, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=b_token_type_ids, 
                            attention_mask=b_input_mask, 
                            labels = b_labels)
    
        
        loss = outputs[0] 
        logits = outputs[1] 
        
        # 로스 구함 
        eval_loss += loss.item() 
        
        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    
    avg_val_loss = eval_loss / len(validation_dataloader)            
    print("  Average validation loss: {0:.2f}".format(avg_val_loss))
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    
    torch.save(model.state_dict(), "ELECTRA_MORE_FEATURES_" + str(epoch_i + 1)) 
print("")
print("Training complete!")



======== Epoch 1 / 30 ========
Training...
  Batch    20  of  46,094.    Elapsed: 0:00:20.
  current average loss = 3.3734192490577697
  Batch    40  of  46,094.    Elapsed: 0:00:40.
  current average loss = 2.8922974944114683
  Batch    60  of  46,094.    Elapsed: 0:00:59.
  current average loss = 2.5170099178949994
  Batch    80  of  46,094.    Elapsed: 0:01:19.
  current average loss = 2.2658097237348556
  Batch   100  of  46,094.    Elapsed: 0:01:39.
  current average loss = 2.0648127949237822
  Batch   120  of  46,094.    Elapsed: 0:01:59.
  current average loss = 1.9048102746407192
  Batch   140  of  46,094.    Elapsed: 0:02:19.
  current average loss = 1.7913209710802351
  Batch   160  of  46,094.    Elapsed: 0:02:39.
  current average loss = 1.6967548161745072
  Batch   180  of  46,094.    Elapsed: 0:02:59.
  current average loss = 1.6278184092707104
  Batch   200  of  46,094.    Elapsed: 0:03:18.
  current average loss = 1.5629123160243035
  Batch   220  of  46,094.    Elapse

  Batch 1,880  of  46,094.    Elapsed: 0:31:12.
  current average loss = 0.8280578042202174
  Batch 1,900  of  46,094.    Elapsed: 0:31:31.
  current average loss = 0.82605315124518
  Batch 1,920  of  46,094.    Elapsed: 0:31:51.
  current average loss = 0.8240391613527511
  Batch 1,940  of  46,094.    Elapsed: 0:32:11.
  current average loss = 0.8213979457917902
  Batch 1,960  of  46,094.    Elapsed: 0:32:31.
  current average loss = 0.8193414914022599
  Batch 1,980  of  46,094.    Elapsed: 0:32:51.
  current average loss = 0.816797198769119
  Batch 2,000  of  46,094.    Elapsed: 0:33:11.
  current average loss = 0.8151788487657905
  Batch 2,020  of  46,094.    Elapsed: 0:33:31.
  current average loss = 0.8136081774045926
  Batch 2,040  of  46,094.    Elapsed: 0:33:51.
  current average loss = 0.8109857597771812
  Batch 2,060  of  46,094.    Elapsed: 0:34:11.
  current average loss = 0.8091670307719592
  Batch 2,080  of  46,094.    Elapsed: 0:34:31.
  current average loss = 0.80768686

  Batch 3,680  of  46,094.    Elapsed: 1:01:08.
  current average loss = 0.6903673172543716
  Batch 3,700  of  46,094.    Elapsed: 1:01:28.
  current average loss = 0.6888334125864345
  Batch 3,720  of  46,094.    Elapsed: 1:01:47.
  current average loss = 0.6872160450244943
  Batch 3,740  of  46,094.    Elapsed: 1:02:07.
  current average loss = 0.6857931097700475
  Batch 3,760  of  46,094.    Elapsed: 1:02:27.
  current average loss = 0.6843452252110743
  Batch 3,780  of  46,094.    Elapsed: 1:02:47.
  current average loss = 0.6831219584873272
  Batch 3,800  of  46,094.    Elapsed: 1:03:07.
  current average loss = 0.6821428263952073
  Batch 3,820  of  46,094.    Elapsed: 1:03:27.
  current average loss = 0.6808914483705272
  Batch 3,840  of  46,094.    Elapsed: 1:03:47.
  current average loss = 0.6795745988434646
  Batch 3,860  of  46,094.    Elapsed: 1:04:07.
  current average loss = 0.6786301830021056
  Batch 3,880  of  46,094.    Elapsed: 1:04:26.
  current average loss = 0.67758

  Batch 5,480  of  46,094.    Elapsed: 1:30:58.
  current average loss = 0.6044515474284761
  Batch 5,500  of  46,094.    Elapsed: 1:31:18.
  current average loss = 0.6034368635443124
  Batch 5,520  of  46,094.    Elapsed: 1:31:38.
  current average loss = 0.6025081567412269
  Batch 5,540  of  46,094.    Elapsed: 1:31:58.
  current average loss = 0.6017765706869024
  Batch 5,560  of  46,094.    Elapsed: 1:32:18.
  current average loss = 0.6011537992536164
  Batch 5,580  of  46,094.    Elapsed: 1:32:38.
  current average loss = 0.6001258125735653
  Batch 5,600  of  46,094.    Elapsed: 1:32:58.
  current average loss = 0.5993875617560531
  Batch 5,620  of  46,094.    Elapsed: 1:33:18.
  current average loss = 0.5985234415191772
  Batch 5,640  of  46,094.    Elapsed: 1:33:38.
  current average loss = 0.5979350353518488
  Batch 5,660  of  46,094.    Elapsed: 1:33:58.
  current average loss = 0.5970394000700632
  Batch 5,680  of  46,094.    Elapsed: 1:34:18.
  current average loss = 0.59627

  Batch 7,280  of  46,094.    Elapsed: 2:00:50.
  current average loss = 0.5471552342473233
  Batch 7,300  of  46,094.    Elapsed: 2:01:10.
  current average loss = 0.5464653270120082
  Batch 7,320  of  46,094.    Elapsed: 2:01:30.
  current average loss = 0.546066128007579
  Batch 7,340  of  46,094.    Elapsed: 2:01:50.
  current average loss = 0.5456826775037869
  Batch 7,360  of  46,094.    Elapsed: 2:02:10.
  current average loss = 0.5451101958478356
  Batch 7,380  of  46,094.    Elapsed: 2:02:30.
  current average loss = 0.5443680596828138
  Batch 7,400  of  46,094.    Elapsed: 2:02:50.
  current average loss = 0.5437201623256142
  Batch 7,420  of  46,094.    Elapsed: 2:03:09.
  current average loss = 0.543173278012687
  Batch 7,440  of  46,094.    Elapsed: 2:03:29.
  current average loss = 0.542661546254831
  Batch 7,460  of  46,094.    Elapsed: 2:03:49.
  current average loss = 0.5422567781929836
  Batch 7,480  of  46,094.    Elapsed: 2:04:09.
  current average loss = 0.54165713

...

  Batch 23,740  of  46,094.    Elapsed: 6:32:40.
  current average loss = 0.019481649557905305
  Batch 23,760  of  46,094.    Elapsed: 6:32:59.
  current average loss = 0.019480319566921228
  Batch 23,780  of  46,094.    Elapsed: 6:33:19.
  current average loss = 0.01946568100496224
  Batch 23,800  of  46,094.    Elapsed: 6:33:39.
  current average loss = 0.019456215211182456
  Batch 23,820  of  46,094.    Elapsed: 6:33:59.
  current average loss = 0.019448913386330872
  Batch 23,840  of  46,094.    Elapsed: 6:34:19.
  current average loss = 0.019446057770885653
  Batch 23,860  of  46,094.    Elapsed: 6:34:39.
  current average loss = 0.01945616425306217
  Batch 23,880  of  46,094.    Elapsed: 6:34:58.
  current average loss = 0.019454366554875844
  Batch 23,900  of  46,094.    Elapsed: 6:35:18.
  current average loss = 0.01945014805684965
  Batch 23,920  of  46,094.    Elapsed: 6:35:38.
  current average loss = 0.019446645836177653
  Batch 23,940  of  46,094.    Elapsed: 6:35:58.
  cu

  Batch 25,480  of  46,094.    Elapsed: 7:01:28.
  current average loss = 0.019489399415003706
  Batch 25,500  of  46,094.    Elapsed: 7:01:47.
  current average loss = 0.019485180167517827
  Batch 25,520  of  46,094.    Elapsed: 7:02:07.
  current average loss = 0.019487815727421365
  Batch 25,540  of  46,094.    Elapsed: 7:02:27.
  current average loss = 0.019481348644755342
  Batch 25,560  of  46,094.    Elapsed: 7:02:47.
  current average loss = 0.01949356487990067
  Batch 25,580  of  46,094.    Elapsed: 7:03:07.
  current average loss = 0.019503259587380525
  Batch 25,600  of  46,094.    Elapsed: 7:03:26.
  current average loss = 0.019541408859877266
  Batch 25,620  of  46,094.    Elapsed: 7:03:46.
  current average loss = 0.01953619765364954
  Batch 25,640  of  46,094.    Elapsed: 7:04:06.
  current average loss = 0.019536476716175442
  Batch 25,660  of  46,094.    Elapsed: 7:04:26.
  current average loss = 0.019535164035848013
  Batch 25,680  of  46,094.    Elapsed: 7:04:46.
  c

  Batch 27,220  of  46,094.    Elapsed: 7:30:15.
  current average loss = 0.01936918292762568
  Batch 27,240  of  46,094.    Elapsed: 7:30:35.
  current average loss = 0.01936108596854065
  Batch 27,260  of  46,094.    Elapsed: 7:30:55.
  current average loss = 0.01936614839715428
  Batch 27,280  of  46,094.    Elapsed: 7:31:15.
  current average loss = 0.0193554939792964
  Batch 27,300  of  46,094.    Elapsed: 7:31:34.
  current average loss = 0.019349427458253646
  Batch 27,320  of  46,094.    Elapsed: 7:31:54.
  current average loss = 0.01933800037520751
  Batch 27,340  of  46,094.    Elapsed: 7:32:14.
  current average loss = 0.019328454661509398
  Batch 27,360  of  46,094.    Elapsed: 7:32:34.
  current average loss = 0.019349755872686378
  Batch 27,380  of  46,094.    Elapsed: 7:32:54.
  current average loss = 0.019336149613308352
  Batch 27,400  of  46,094.    Elapsed: 7:33:14.
  current average loss = 0.019324904457074556
  Batch 27,420  of  46,094.    Elapsed: 7:33:34.
  curre

  Batch 28,960  of  46,094.    Elapsed: 7:59:02.
  current average loss = 0.01937005128229078
  Batch 28,980  of  46,094.    Elapsed: 7:59:22.
  current average loss = 0.019378868129819097
  Batch 29,000  of  46,094.    Elapsed: 7:59:42.
  current average loss = 0.0193792222176261
  Batch 29,020  of  46,094.    Elapsed: 8:00:02.
  current average loss = 0.01938112596227055
  Batch 29,040  of  46,094.    Elapsed: 8:00:22.
  current average loss = 0.019374217623883225
  Batch 29,060  of  46,094.    Elapsed: 8:00:42.
  current average loss = 0.019369582867048582
  Batch 29,080  of  46,094.    Elapsed: 8:01:02.
  current average loss = 0.019360193170855612
  Batch 29,100  of  46,094.    Elapsed: 8:01:21.
  current average loss = 0.019351182538324716
  Batch 29,120  of  46,094.    Elapsed: 8:01:41.
  current average loss = 0.019350790244509253
  Batch 29,140  of  46,094.    Elapsed: 8:02:01.
  current average loss = 0.019358939272795812
  Batch 29,160  of  46,094.    Elapsed: 8:02:21.
  cur

KeyboardInterrupt: 

# Preprocess Test Data and Predict

똑같이 Test Data를 sliding window로 잘라주고, tokenization을 해준다. 

Voting 기법을 사용할 예정이다. 

Sample 하나가 들어오면 sliding window로 나누어서 총 N개의 subtext들로 나눈다. 

각 N개의 subtext에 대해서 예측을 해준다. 그리고 예측된 확률값들의 평균을 구한다. 이후 확률의 평균의 argmax로 클래스를 정한다. 

이러한 soft-voting이 hard-voting보다 나을것이라고 생각된다. 

In [58]:
# load best checkpoint model
checkpoint = torch.load('ELECTRA_MORE_FEATURES_6') 
test_model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=46)
test_model.load_state_dict(checkpoint) 
test_model.cuda() 

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [59]:
import torch.nn.functional as nnf

test['사업명'].fillna('NAN',inplace=True) 
test['사업_부처명'].fillna('NAN',inplace=True) 
test['내역사업명'].fillna('NAN',inplace=True) 
test['과제명'].fillna('NAN',inplace=True) 
test['요약문_한글키워드'].fillna('NAN',inplace=True)

In [71]:
test_contents = test['요약문_내용'].values 
test_feature1 = test['사업명'].values 
test_feature2 = test['사업_부처명'].values 
test_feature3 = test['내역사업명'].values 
test_feature4 = test['과제명'].values 
test_feature5 = test['요약문_한글키워드'].values 

predicted_classes = []

# change to eval mode 
test_model.eval() 

for i in tqdm(range(test_contents.shape[0]), position=0, leave=True):
    f1 = clean_text(str(test_feature1[i])) 
    f2 = clean_text(str(test_feature2[i])) 
    f3 = clean_text(str(test_feature3[i])) 
    f4 = clean_text(str(test_feature4[i])) 
    f5 = str(test_feature5[i]) 
    splitted = split_text(str(test_contents[i])) 
    # make predictions for each splitted text 
    probabilities = [] 
    for text in splitted: 
        test_text = f1 + " " + f2 + " " + f3 + " " + f4 + " " + f5 + " " + text 
        # tokenize test text 
        input_id, attention_mask, token_type_id = electra_tokenizer(test_text, MAX_LEN=MAX_LEN) 
        input_id = torch.tensor(input_id)
        attention_mask = torch.tensor(attention_mask) 
        token_type_id = torch.tensor(token_type_id) 
        # reshape into (batch, MAX_LEN)
        input_id = torch.reshape(input_id, (-1,MAX_LEN)) 
        attention_mask = torch.reshape(attention_mask, (-1,MAX_LEN)) 
        token_type_id = torch.reshape(token_type_id, (-1,MAX_LEN)) 
        # move tensor to cuda 
        input_id = input_id.to(device) 
        attention_mask = attention_mask.to(device) 
        token_type_id = token_type_id.to(device) 
        
        with torch.no_grad(): 
            outputs = test_model(input_id, 
                            token_type_ids=token_type_id, 
                            attention_mask=attention_mask) 
        
        logits = outputs[0]
        
        # obtain softmax probabilities 
        prob = nnf.softmax(logits, dim=1).flatten()
        probabilities.append(prob)
    
    # soft voting 
    prob_sum = np.zeros(46) 
    for i in range(len(probabilities)): 
        for j in range(46): 
            prob_sum[j] += probabilities[i][j] 
            
    
    prob_sum /= len(probabilities)  
    
    predicted_class = np.argmax(prob_sum)
    
    predicted_classes.append(predicted_class)
    
    

  7%|▋         | 3140/43576 [10:28<2:26:37,  4.60it/s]

Long Text!! Using Head+Tail Truncation


 14%|█▎        | 5920/43576 [19:39<1:18:45,  7.97it/s]

Long Text!! Using Head+Tail Truncation


 20%|█▉        | 8594/43576 [28:09<2:20:23,  4.15it/s]

...

 87%|████████▋ | 38030/43576 [2:02:05<20:42,  4.46it/s]

Long Text!! Using Head+Tail Truncation


 92%|█████████▏| 40017/43576 [2:08:39<08:06,  7.31it/s]

Long Text!! Using Head+Tail Truncation


100%|██████████| 43576/43576 [2:20:15<00:00,  5.18it/s]


In [72]:
submission.iloc[:,1] = predicted_classes 

submission 

,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,0
4,174308,0
...,...,...
43571,217875,0
43572,217876,0
43573,217877,2
43574,217878,0


In [73]:
submission.to_csv("ELECTRA_VOTING.csv",index=False) 